In [1]:
from finetune import Workspace as W
import os
import hydra
from omegaconf import OmegaConf
import numpy as np

/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


In [2]:
import pathlib
import hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()
config_dir = pathlib.Path('./')
hydra.initialize(config_path=config_dir)

cfg = hydra.compose(config_name='finetune.yaml', overrides=[])


In [3]:
cfg
workspace = W(cfg)

workspace: /home/seolho/workplace/self-supervised-rl/for commit/cic


In [4]:
np.random.uniform(0,1,64).astype(np.float32)
skill = np.ones(64).astype(np.float32)
#workspace.eval(skill)

In [5]:
workspace.gather_trajectories(skill, 2000)
len(workspace.replay_storage)

2000

In [8]:
workspace.agent.behavior_cloning(workspace.replay_iter, 0)

IndexError: Caught IndexError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/seolho/anaconda3/envs/urlb/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 32, in fetch
    data.append(next(self.dataset_iter))
  File "/home/seolho/workplace/self-supervised-rl/for commit/cic/replay_buffer.py", line 169, in __iter__
    yield self._sample()
  File "/home/seolho/workplace/self-supervised-rl/for commit/cic/replay_buffer.py", line 150, in _sample
    episode = self._sample_episode()
  File "/home/seolho/workplace/self-supervised-rl/for commit/cic/replay_buffer.py", line 99, in _sample_episode
    eps_fn = random.choice(self._episode_fns)
  File "/home/seolho/anaconda3/envs/urlb/lib/python3.8/random.py", line 290, in choice
    raise IndexError('Cannot choose from an empty 

In [31]:
workspace.agent.update(workspace.replay_iter, 0)

{'batch_reward': 0.13910725712776184,
 'critic_target_q': 0.17433397471904755,
 'critic_q1': 0.1872975081205368,
 'critic_q2': 0.19648459553718567,
 'critic_loss': 0.27787157893180847,
 'actor_loss': -0.21502485871315002,
 'actor_logprob': 2.795954942703247,
 'actor_ent': -1.1429965496063232}

In [39]:
workspace.train_env.observation_spec()

Array(shape=(24,), dtype=dtype('float32'), name='observation')

In [33]:
next(workspace.replay_iter)[0].shape

torch.Size([1024, 24])

3000

In [ ]:
metrics = self.agent.update(self.replay_iter, self.global_step)

In [4]:
workspace.train()

size :  1
size :  1
size :  1
size :  1
size :  1
size :  1
size :  1
size :  1
size :  1
size :  1
| eval  | F: 0 | S: 0 | E: 0 | L: 1000 | R: 244.6752 | T: 0:00:00


TypeError: init_meta() missing 1 required positional argument: 'skill'

In [11]:
class Until:
    def __init__(self, until, action_repeat=1):
        self._until = until
        self._action_repeat = action_repeat

    def __call__(self, step):
        if self._until is None:
            return True
        until = self._until // self._action_repeat
        return step < until

In [12]:
t = Until(10)

0
1
2
3
4
5
6
7
8
9


In [ ]:
    def behavior_cloning(self, replay_iter, step):
        metrics = dict()

        if step % self.update_every_steps != 0:
            return metrics

        batch = next(replay_iter)

        obs, action, reward, discount, next_obs, skill = utils.to_torch(
            batch, self.device)

        with torch.no_grad():
            obs = self.aug_and_encode(obs)
        
            next_obs = self.aug_and_encode(next_obs)


        if self.use_tb or self.use_wandb:
            metrics['batch_reward'] = reward.mean().item()

        # extend observations with skill
        obs = torch.cat([obs, skill], dim=1)
        next_obs = torch.cat([next_obs, skill], dim=1)
        
        stddev = utils.schedule(self.stddev_schedule, step)
        dist = self.actor(obs, stddev)
        agent_action = dist.sample(clip=self.stddev_clip)
        
        bc_loss = torch.mean((action - agent_action)**2)
        
        self.actor_opt.zero_grad(set_to_none=True)
        loss.backward()
        self.actor_opt.step()
        # update actor
        metrics['bc_loss'] = bc_loss.item()

        return metrics
